In [19]:
import os
import pandas as pd
import zipfile
import pyodbc
import requests
import io
from datetime import datetime, timedelta
import sqlalchemy
import urllib
import warnings

warnings.filterwarnings('ignore')

In [22]:

#funciones

def iterar_entre_fechas(fecha_desde, fecha_hasta):
    fecha_actual = datetime.strptime(fecha_desde, "%Y-%m-%dT%H:%M:%S.%f%z")
    fecha_fin = datetime.strptime(fecha_hasta, "%Y-%m-%dT%H:%M:%S.%f%z")

    # Asegurarse de que fecha_actual sea exactamente a la medianoche
    fecha_actual = fecha_actual.replace(hour=0, minute=0, second=0, microsecond=0)

    while fecha_actual <= fecha_fin:
        fecha_siguiente = fecha_actual + timedelta(hours=23, minutes=59)
        yield fecha_actual, fecha_siguiente
        # Añadir un día para la próxima iteración
        fecha_actual += timedelta(days=1)
        
#Con este script se puede obtener la fecha del último documento cargado por CAMMESA
def  ultimo_dia_CAMM(nemo:str):

    ultimafecha = requests.get(f"https://api.cammesa.com/pub-svc/public/obtieneFechaUltimoDocumento?nemo={nemo}")

    fecha = ultimafecha.text[10:-2]

    return fecha

def ultimo_dia(tabla:str):
    server = 'DARCCVWSQL19'
    database = 'TAPI'
    tabla = tabla

    connection_string = f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes;'

    # Conectar a la base de datos
    connection = pyodbc.connect(connection_string)

    # Crear un cursor para ejecutar consultas
    cursor = connection.cursor()

    # TODO: Crear acá la tabla en la BBDD
    #  Consulta SQL
    query = f"SELECT TOP 1 FECHA FROM {tabla} ORDER BY FECHA DESC"

    # Ejecutar la consulta
    df = pd.read_sql(query, connection)

    # Cerrar el cursor
    cursor.close()

    ultimo_dia = df['FECHA'][0]
    
    return ultimo_dia

In [29]:
#TODO: testeando fecha ultimo dia del mes para los corregidos

def es_ultimo_dia_del_mes(fecha):
    siguiente_mes = fecha.replace(day=28) + timedelta(days=4)  # Pasamos al día 28 y sumamos 4 días
    ultimo_dia = siguiente_mes - timedelta(days=siguiente_mes.day)
    return fecha.day == ultimo_dia.day

# Obtener la fecha actual
fecha_now = datetime.now()

print(es_ultimo_dia_del_mes())

# Verificar si es el último día del mes
if es_ultimo_dia_del_mes(fecha_now):
    print("Es el último día del mes.")
else:
    print("No es el último día del mes.")

<function es_ultimo_dia_del_mes at 0x0000019A57B7C540>
No es el último día del mes.


In [ ]:
NEMO_CORREGIDOS = "PARTE_POST_OPERATIVO_UNIF"
NEMO_NO_CORREGIDOS = "PARTE_POST_OPERATIVO"

In [25]:
print(ultimo_dia("Valores_Corregidos_Test"))
print(ultimo_dia_CAMM("PARTE_POST_OPERATIVO_UNIF"))

2024-05-31
2024-06-12


In [26]:
# DATOS DE LA BBDD, SERVER Y TABLA

server = 'DARCCVWSQL19'
database = 'TAPI'

#1
#-------------------------------------------------------#
tabla_valores = 'Valores_Corregidos_Test'
#tabla_contratos = 'Contratos_Corregidos'
#tabla_novedades = 'Novedades_Corregidos'

#-------------------------------------------------------#

#2
#-------------------------------------------------------#
# Fechas para seleccionar el día de la carga se debe iterar

# fecha_desde_obj = datetime.fromisoformat(ultimo_dia_bd())

# fecha_hasta_obj = datetime.fromisoformat(ultimo_dia_CAMM())

# fecha_desde_bd = ultimo_dia_bd() #OJO DE QUE TABLA TOMA EL ULTIMO DIA

# fecha_datetime = datetime.strptime(fecha_desde_bd, "%Y-%m-%d")

# fecha_siguiente = fecha_datetime + timedelta(days=1) #Sumar un día

# fecha_desde = fecha_siguiente.strftime("%Y-%m-%d") #Convertir de nuevo a string si es necesario

# fecha_hasta = ultimo_dia_CAMM() #ultimo informe en API CAMMESA

# fecha_desde = fecha_desde+"T00:00:00.000-03:00" 
# fecha_hasta = fecha_hasta+"T23:59:00.000-03:00"

# fecha_desde_datetime = datetime.strptime(fecha_desde)
# fecha_hasta_datetime = datetime.strptime(fecha_hasta)

#fecha_desde = "2024-04-20T00:00:00.000-03:00"
#fecha_hasta = "2024-04-30T23:59:59.000-03:00"

#Defino la tabla de CAMMESA que me voy a traer

NEMO_CORREGIDOS = "PARTE_POST_OPERATIVO_UNIF"
NEMO_NO_CORREGIDOS = "PARTE_POST_OPERATIVO"
#-------------------------------------------------------#

#URL para capturar Id del documento y el zip file:

#Busco los zip disponibles para traer así puedo extraer el id
URL = f"https://api.cammesa.com/pub-svc/public/"

method_id = "findDocumentosByNemoRango?" #ID
method_zip = "findAllAttachmentZipByNemoId?" #metodo


zip_path = r".zips"
mdb_path = r".zips\.mdb"

connection_string = f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Establecer la conexión con la base de datos de SQL Server
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

#url_doc_id = f"{URL}{method_id}fechadesde={fecha_desde}&fechahasta={fecha_hasta}&nemo={NEMO}"

dataframes = []
dfout = pd.DataFrame()
dfout2 = pd.DataFrame()
df_filtrado = pd.DataFrame()

In [27]:
fecha_desde = "2024-05-01T00:00:00.000-03:00"
fecha_hasta = "2024-05-31T23:59:59.000-03:00"
fecha_desde_obj = 0
fecha_hasta_obj = 1

fecha_ultimo_dia_mes = ""

In [6]:

def principal_no_corregidos():

    if fecha_desde_obj < fecha_hasta_obj:
        print("Se realiza el update")

        for fecha_actual, fecha_siguiente in iterar_entre_fechas(fecha_desde, fecha_hasta):
            
            #inicializo los 3 dataframes del proceso
            valores_generadores = pd.DataFrame()
            contrato_abastecimiento = pd.DataFrame()
            novedades = pd.DataFrame()

            #url para obtener el doc id de las fecha de la iteracion
            url_doc_id = f"{URL}{method_id}fechadesde={fecha_actual.isoformat()}&fechahasta={fecha_siguiente.isoformat()}&nemo={NEMO_NO_CORREGIDOS}"
            
            #obtener el doc_id del dia actual (corregido)
            dia_mdb = fecha_actual.strftime("%d-%m-%Y")
            try:
                with requests.get(url_doc_id) as response:
                    if response.status_code == 200:
                        PPO=response.json()
                        doc_id = PPO[-1]['id']
                        print(dia_mdb)
                    else:
                        print("La solicitud falló con el código de estado:", response.status_code)
            except requests.exceptions.RequestException as e:
                # Manejar la excepción
                print("Error al realizar la solicitud:", e)


            #url para generar el .zip del doc id del día de la iteracion
            url_zip = f"{URL}{method_zip}docId={doc_id}&nemo={NEMO_NO_CORREGIDOS}"


            #descargar el .zip del doc_id (corregido)
            try:
                with requests.get(url_doc_id) as response:
                    if response.status_code == 200:
                        r = requests.get(url_zip)

                        # Crear un objeto ZipFile a partir del contenido descargado
                        z = zipfile.ZipFile(io.BytesIO(r.content))

                        # Directorio de destino para extraer los archivos ZIP
                        destination_directory = ".zips"

                        # Extraer todos los archivos del ZIP en el directorio específico
                        z.extractall(destination_directory)
                        zip_name = z.namelist()[0]
                    else:
                        print("La solicitud falló con el código de estado:", response.status_code)

            except requests.exceptions.RequestException as e:
                # Manejar la excepción
                print("Error al realizar la solicitud:", e)
            
            #Colocar los PATHs correctos donde se traeran los archivos
            
            path_zip_dia = f"{zip_path}\{zip_name}"

            #display(path_zip_dia)

            
            try:
                # Extrae el archivo MDB de cada archivo ZIP diario
                with zipfile.ZipFile(path_zip_dia, 'r') as zip_ref:
                    # Encontrar el nombre del archivo MDB dentro del ZIP diario
                    archivo_mdb = os.path.splitext(zip_name)[0] + ".mdb"
                    zip_ref.extract(archivo_mdb, path=mdb_path)

        
                # Lee el archivo MDB y cargar la tabla VALORES_GENERADORES en un dataframe
                mdb_file = os.path.join(mdb_path, archivo_mdb)
                conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={mdb_file};"
                conn = pyodbc.connect(conn_str)

                #3
                #-------------------------------------------------------#
                valores_generadores = pd.read_sql("SELECT * FROM VALORES_GENERADORES", conn)
                #contrato_abastecimiento = pd.read_sql("SELECT * FROM CONTRATO_ABASTECIMIENTO", conn)
                #novedades = pd.read_sql("SELECT * FROM NOVEDADES", conn)
                #-------------------------------------------------------#
                conn.close()
                
                # Convertir dia_mdb a un objeto datetime
                dia_datetime = datetime.strptime(dia_mdb, '%d-%m-%Y')

                # Formatear la fecha en el formato YYYY-MM-DD como una cadena
                dia_mdb_formatted = dia_datetime.strftime('%Y-%m-%d')

                # Insertar la fecha formateada en la lista valores_generadores
                
                #4
                #-------------------------------------------------------#
                valores_generadores.insert(0, 'FECHA', dia_mdb_formatted)

                #contrato_abastecimiento.insert(0, 'FECHA', dia_mdb_formatted)

                #novedades.insert(0, 'FECHA', dia_mdb_formatted)
                #-------------------------------------------------------#
                quoted = urllib.parse.quote_plus(connection_string)

                #Por limitaciones de tamaño de excel filtramos solo las máquinas Pampa
                valores_filtrados = ["ADTOHI", "AR21EO", "BAHIEO", "BBLATV29", "BBLATV30",
                                    "BBLMDI01", "BBLMDI02", "BBLMDI03", "BBLMDI04", 
                                    "BBLMDI05", "BBLMDI06", "CERITV01", "CORTEO", 
                                    "EBARTG01", "EBARTG02", "EBARTV01", "ETIGHI", 
                                    "GEBATG01", "GEBATG02", "GEBATG03", "GEBATG04", 
                                    "GEBATV01", "GEBATV02", "GUEMTG01", "GUEMTV11", 
                                    "GUEMTV12", "GUEMTV13", "LDLATG01", "LDLATG02", 
                                    "LDLATG03", "LDLATG04", "LDLATG05", "LDLATV01", 
                                    "LDLMDI01", "LREYHB", "NIH1HI", "NIH2HI", "NIH3HI", 
                                    "PAMEEO","PE32EO", "PEP3EO", "PILBDI01", "PILBDI02", 
                                    "PILBDI03", "PILBDI04", "PILBDI05", "PILBDI06", "PIQIDI01", "PPLEHI"]
                
                contratos_filtrados = ["C.T. LOMA DE LA LATA", "C.T.E.BARRAGAN TV-M", "CT LOMA II LA LATA-M", "GENELBA CC -MERCA", "PIEDRABUENA  R21-","CT PILAR BS AS M"]

                # Filtrar el DataFrame por los valores especificados en la columna "GRUPO"

                #DEFINIR EL DF A SUBIR

                #5
                #-------------------------------------------------------#

                df_valores = valores_generadores[valores_generadores["GRUPO"].isin(valores_filtrados)]  
                
                #df_contratos = contrato_abastecimiento[contrato_abastecimiento["CONTRATO"].isin(contratos_filtrados)]

                #df_novedades = novedades[novedades["GRUPO"].isin(valores_filtrados)]

                #-------------------------------------------------------#

                #TODO: Revisar el filtrado previo a igresar la data    
                
                #df_filtrado = df_filtrado.sort_values(by=['FECHA', 'HORA', 'GRUPO'])

                engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
            
                #ACÁ ESCRIBE LA DATA, REVISAR SIEMPRE ARGUMENTOS

                #6
                #-------------------------------------------------------#

                df_valores.to_sql(f'{tabla_valores}', schema='dbo', con=engine, if_exists='append', chunksize=20000)

                #df_contratos.to_sql(f'{tabla_contratos}', schema='dbo', con=engine, if_exists='append', chunksize=20000)

                #df_novedades.to_sql(f'{tabla_novedades}', schema='dbo', con=engine, if_exists='append', chunksize=20000)

                #dfout = pd.concat([dfout,df_valores], ignore_index=True)
                #dfout2 = pd.concat([dfout2, df_contratos], ignore_index=True)

                #-------------------------------------------------------#
                
            except FileNotFoundError:
                print(f"El archivo {zip_name} no se encontró. Saltando al siguiente archivo...")

    else:
        print(f"Última fecha en BD:{fecha_desde_obj} es igual a la última fecha del informe de CAMMESA: {fecha_hasta_obj}\nNo se realiza el update")

    print("Finaliza la carga de No Corregidos")

def principal_corregidos():

    if fecha_desde_obj < fecha_hasta_obj:
        print("Se realiza el update de Corregidos")

        for fecha_actual, fecha_siguiente in iterar_entre_fechas(fecha_desde, fecha_hasta):
            
            #inicializo los 3 dataframes del proceso
            valores_generadores = pd.DataFrame()
            contrato_abastecimiento = pd.DataFrame()
            novedades = pd.DataFrame()

            #url para obtener el doc id de las fecha de la iteracion
            url_doc_id = f"{URL}{method_id}fechadesde={fecha_actual.isoformat()}&fechahasta={fecha_siguiente.isoformat()}&nemo={NEMO_CORREGIDOS}"
            
            #obtener el doc_id del dia actual (corregido)
            dia_mdb = fecha_actual.strftime("%d-%m-%Y")
            try:
                with requests.get(url_doc_id) as response:
                    if response.status_code == 200:
                        PPO=response.json()
                        doc_id = PPO[-1]['id']
                        print(dia_mdb)
                    else:
                        print("La solicitud falló con el código de estado:", response.status_code)
            except requests.exceptions.RequestException as e:
                # Manejar la excepción
                print("Error al realizar la solicitud:", e)


            #url para generar el .zip del doc id del día de la iteracion
            url_zip = f"{URL}{method_zip}docId={doc_id}&nemo={NEMO_CORREGIDOS}"


            #descargar el .zip del doc_id (corregido)
            try:
                with requests.get(url_doc_id) as response:
                    if response.status_code == 200:
                        r = requests.get(url_zip)

                        # Crear un objeto ZipFile a partir del contenido descargado
                        z = zipfile.ZipFile(io.BytesIO(r.content))

                        # Directorio de destino para extraer los archivos ZIP
                        destination_directory = ".zips"

                        # Extraer todos los archivos del ZIP en el directorio específico
                        z.extractall(destination_directory)
                        zip_name = z.namelist()[0]
                    else:
                        print("La solicitud falló con el código de estado:", response.status_code)

            except requests.exceptions.RequestException as e:
                # Manejar la excepción
                print("Error al realizar la solicitud:", e)
            
            #Colocar los PATHs correctos donde se traeran los archivos
            
            path_zip_dia = f"{zip_path}\{zip_name}"

            #display(path_zip_dia)

            
            try:
                # Extrae el archivo MDB de cada archivo ZIP diario
                with zipfile.ZipFile(path_zip_dia, 'r') as zip_ref:
                    # Encontrar el nombre del archivo MDB dentro del ZIP diario
                    archivo_mdb = os.path.splitext(zip_name)[0] + ".mdb"
                    zip_ref.extract(archivo_mdb, path=mdb_path)

        
                # Lee el archivo MDB y cargar la tabla VALORES_GENERADORES en un dataframe
                mdb_file = os.path.join(mdb_path, archivo_mdb)
                conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={mdb_file};"
                conn = pyodbc.connect(conn_str)

                #3
                #-------------------------------------------------------#
                valores_generadores = pd.read_sql("SELECT * FROM VALORES_GENERADORES", conn)
                #contrato_abastecimiento = pd.read_sql("SELECT * FROM CONTRATO_ABASTECIMIENTO", conn)
                #novedades = pd.read_sql("SELECT * FROM NOVEDADES", conn)
                #-------------------------------------------------------#
                conn.close()
                
                # Convertir dia_mdb a un objeto datetime
                dia_datetime = datetime.strptime(dia_mdb, '%d-%m-%Y')

                # Formatear la fecha en el formato YYYY-MM-DD como una cadena
                dia_mdb_formatted = dia_datetime.strftime('%Y-%m-%d')

                # Insertar la fecha formateada en la lista valores_generadores
                
                #4
                #-------------------------------------------------------#
                valores_generadores.insert(0, 'FECHA', dia_mdb_formatted)

                #contrato_abastecimiento.insert(0, 'FECHA', dia_mdb_formatted)

                #novedades.insert(0, 'FECHA', dia_mdb_formatted)
                #-------------------------------------------------------#
                quoted = urllib.parse.quote_plus(connection_string)

                #Por limitaciones de tamaño de excel filtramos solo las máquinas Pampa
                valores_filtrados = ["ADTOHI", "AR21EO", "BAHIEO", "BBLATV29", "BBLATV30",
                                    "BBLMDI01", "BBLMDI02", "BBLMDI03", "BBLMDI04", 
                                    "BBLMDI05", "BBLMDI06", "CERITV01", "CORTEO", 
                                    "EBARTG01", "EBARTG02", "EBARTV01", "ETIGHI", 
                                    "GEBATG01", "GEBATG02", "GEBATG03", "GEBATG04", 
                                    "GEBATV01", "GEBATV02", "GUEMTG01", "GUEMTV11", 
                                    "GUEMTV12", "GUEMTV13", "LDLATG01", "LDLATG02", 
                                    "LDLATG03", "LDLATG04", "LDLATG05", "LDLATV01", 
                                    "LDLMDI01", "LREYHB", "NIH1HI", "NIH2HI", "NIH3HI", 
                                    "PAMEEO","PE32EO", "PEP3EO", "PILBDI01", "PILBDI02", 
                                    "PILBDI03", "PILBDI04", "PILBDI05", "PILBDI06", "PIQIDI01", "PPLEHI"]
                
                contratos_filtrados = ["C.T. LOMA DE LA LATA", "C.T.E.BARRAGAN TV-M", "CT LOMA II LA LATA-M", "GENELBA CC -MERCA", "PIEDRABUENA  R21-","CT PILAR BS AS M"]

                # Filtrar el DataFrame por los valores especificados en la columna "GRUPO"

                #DEFINIR EL DF A SUBIR

                #5
                #-------------------------------------------------------#

                df_valores = valores_generadores[valores_generadores["GRUPO"].isin(valores_filtrados)]  
                
                #df_contratos = contrato_abastecimiento[contrato_abastecimiento["CONTRATO"].isin(contratos_filtrados)]

                #df_novedades = novedades[novedades["GRUPO"].isin(valores_filtrados)]

                #-------------------------------------------------------#

                #TODO: Revisar el filtrado previo a igresar la data    
                
                #df_filtrado = df_filtrado.sort_values(by=['FECHA', 'HORA', 'GRUPO'])

                engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
            
                #ACÁ ESCRIBE LA DATA, REVISAR SIEMPRE ARGUMENTOS

                #6
                #-------------------------------------------------------#

                df_valores.to_sql(f'{tabla_valores}', schema='dbo', con=engine, if_exists='append', chunksize=20000)

                #df_contratos.to_sql(f'{tabla_contratos}', schema='dbo', con=engine, if_exists='append', chunksize=20000)

                #df_novedades.to_sql(f'{tabla_novedades}', schema='dbo', con=engine, if_exists='append', chunksize=20000)

                #dfout = pd.concat([dfout,df_valores], ignore_index=True)
                #dfout2 = pd.concat([dfout2, df_contratos], ignore_index=True)

                #-------------------------------------------------------#
                
            except FileNotFoundError:
                print(f"El archivo {zip_name} no se encontró. Saltando al siguiente archivo...")

    else:
        print(f"Última fecha en BD:{fecha_desde_obj} es igual a la última fecha del informe de CAMMESA: {fecha_hasta_obj}\nNo se realiza el update")

    print("Finaliza la carga de Corregidos")

if __name__ == "__main__":
    now = datetime.now()
    print(f"Se ejecuta el día {now}")
    print(f"Se ejecuta funcion principal_no_corregidos")
    principal_no_corregidos()
    print(f"Se ejecuta funcion principal_corregidos")
    principal_corregidos()




Se ejecuta el día 2024-06-14 14:03:56.664653
Se realiza el update
01-05-2024


C:\Users\jadurian\AppData\Local\Temp\ipykernel_4204\1865625304.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  valores_generadores = pd.read_sql("SELECT * FROM VALORES_GENERADORES", conn)


02-05-2024


C:\Users\jadurian\AppData\Local\Temp\ipykernel_4204\1865625304.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  valores_generadores = pd.read_sql("SELECT * FROM VALORES_GENERADORES", conn)


03-05-2024


...

30-05-2024


C:\Users\jadurian\AppData\Local\Temp\ipykernel_4204\1865625304.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  valores_generadores = pd.read_sql("SELECT * FROM VALORES_GENERADORES", conn)


31-05-2024


C:\Users\jadurian\AppData\Local\Temp\ipykernel_4204\1865625304.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  valores_generadores = pd.read_sql("SELECT * FROM VALORES_GENERADORES", conn)


Finaliza la carga
